In [27]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

pd.set_option('future.no_silent_downcasting', True)
# DF
df = pd.read_csv("/kaggle/input/connect-4-uci/connect-4.data")

df = df.replace(['x','o','b'],[1,-1,0])
df = df.replace(['loss','draw','win'],[0,1,2])

new_names = [f'feature_{i}' for i in range(42)] + ['label']
df.columns = new_names

X = np.array(df)
Y = X[:,42]
X = X[:,0:42]
# X is the Data for training along with Y as labels

# Reshape for CNN
X = X.reshape(-1, 6, 7, 1)
print(X.shape)

# Train Test Split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=100)
X_test, X_val, Y_test, Y_val = train_test_split(X_test,Y_test,test_size=0.5,random_state=100)

(67556, 6, 7, 1)
(54044, 6, 7, 1)
(6756, 6, 7, 1)
(6756, 6, 7, 1)
(6756,)
(54044,)
(6756,)


In [33]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# CNN Model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(6, 7, 1), padding='same'),
    MaxPooling2D((2, 2)),
    
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(3, activation='softmax')
])

# Compiling the model
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

X_train = np.array(X_train, dtype=np.float32)
X_test = np.array(X_test, dtype=np.float32)
X_val   = np.array(X_val, dtype=np.float32)
Y_train = np.array(Y_train, dtype=np.float32)
Y_val   = np.array(Y_val, dtype=np.float32)
Y_test   = np.array(Y_test, dtype=np.float32)

# Train the model
history = model.fit(
    X_train, Y_train,
    validation_data=(X_val, Y_val),
    epochs=15,
    batch_size=64
)

# Testing the model
test_loss, test_acc = model.evaluate(X_test, Y_test)
print(f"Test Accuracy: {test_acc:.4f}")

# Save the model
model.save("/kaggle/working/connect4_cnn_model.h5")

Epoch 1/15
845/845 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - accuracy: 0.7179 - loss: 0.7014 - val_accuracy: 0.7577 - val_loss: 0.5742
Epoch 2/15
845/845 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.7813 - loss: 0.5500 - val_accuracy: 0.7975 - val_loss: 0.4969
Epoch 3/15
845/845 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.8058 - loss: 0.4930 - val_accuracy: 0.8141 - val_loss: 0.4672
Epoch 4/15
845/845 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.8195 - loss: 0.4571 - val_accuracy: 0.8246 - val_loss: 0.4394
Epoch 5/15
845/845 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.8270 - loss: 0.4329 - val_accuracy: 0.8326 - val_loss: 0.4246
Epoch 6/15
845/845 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.8345 - loss: 0.4106 - val_accuracy: 0.8321 - val_loss: 0.4164
Epoch 7/15
845/845 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.8409 - loss: 0.3971 - val_accuracy: 0.8298 - val_loss: 0.4153
Epoch 8/15
845/845 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.8493 - loss: 0.3793 - val_accuracy: 0.

In [136]:
x=7
y=6
from tensorflow.keras.models import load_model
import numpy as np

game = [[0 for i in range(x)]for i in range(y)]

def printgame():
    for i in game:
        for j in i:
            print(j,end="  ")
        print()

def drop_piece(a):
    p = int(input("Enter place from 1 to 7 : "))
    for i in range(y):
        if game[y-i-1][p-1]==0 :
            game[y-i-1][p-1] = a
        
            if check_win(p-1, y-i-1, a):
                print(f"Player {a} wins!")
                return True
                
            return False
            
    print(f"Column {p} full, try again")
    return drop_piece(a)

def check_win(col, row, player):
    directions = [
        (0, 1),
        (1, 0),
        (1, 1),
        (1, -1)
    ]
    for dr, dc in directions:
        count = 1
        r, c = row + dr, col + dc
        while 0 <= r < y and 0 <= c < x and game[r][c] == player:
            count += 1
            r += dr
            c += dc
        r, c = row - dr, col - dc
        while 0 <= r < y and 0 <= c < x and game[r][c] == player:
            count += 1
            r -= dr
            c -= dc
        if count >= 4:
            return True
            
    return False
    
def is_board_full():
    return all(game[0][col] != 0 for col in range(x))

def play_game():
    current_player = 1
    print("Connect 4 Game!")
    print("Players are 1 and 2")
    printgame()
    
    while True:
        print(f"\nPlayer {current_player}'s turn:")
        
        if drop_piece(current_player):
            printgame()
            break
            
        if is_board_full():
            print("Board is full! It's a tie!")
            printgame()
            break
            
        printgame()
        current_player = 2 if current_player == 1 else 1

def ai_move(player,model):
    best_score = -1
    best_col = None
    for col in range(x):
        # Check if column is valid
        if game[0][col] != 0:
            continue
        # Copy board and simulate move
        temp_board = [row.copy() for row in game]
        for row in range(y-1, -1, -1):
            if temp_board[row][col] == 0:
                temp_board[row][col] = player
                break
        # Convert to numpy array and reshape for CNN
        input_board = np.array(temp_board).reshape(1,6,7,1)
        pred = model.predict(input_board, verbose=0)
        win_prob = pred[0][2]  # probability of "win"
        if win_prob > best_score:
            best_score = win_prob
            best_col = col
    # Drop piece in best column
    for row in range(y-1, -1, -1):
        if game[row][best_col] == 0:
            game[row][best_col] = player
            return check_win(best_col, row, player)

def play_game_ai():
    current_player = 1  # Human starts
    print("Connect 4 Game!")
    print("Player 1 = Human, Player 2 = AI")
    printgame()
    
    model = load_model("/kaggle/input/connect4cnn-ver1-84.58/tensorflow2/default/1/connect4_cnn_model.h5")
    print("Model loaded successfully!")
    
    while True:
        print(f"\nPlayer {current_player}'s turn:")
        
        if current_player == 1:
            if drop_piece(current_player):  # Human move
                printgame()
                print("Human wins!")
                break
        else:
            if ai_move(current_player,model):  # AI move
                printgame()
                print("AI wins!")
                break
                
        if is_board_full():
            print("Board is full! It's a tie!")
            printgame()
            break
        
        printgame()
        current_player = 2 if current_player == 1 else 1
def reset_game():
    global game
    game = [[0 for _ in range(x)] for _ in range(y)]


In [ ]:
reset_game()
play_game_ai()